In [1]:
import pandas as pd
import requests
import json

### get call demand info

In [79]:
def get_address_by_lat_lng(lat_lng_list):
    headers = {"Content-type" :"application/json"}

    query_params = []
    for i in range(0, len(lat_lng_list), 2):
        query_param = f"locations={','.join(map(str,lat_lng_list[i:i+2]))}"
        query_params.append(query_param)

    facility_url = f"http://navi-facility-api.dev.onkakao.net/region/v1/hcodes?{'&'.join(query_params)}"

    resp = requests.get(facility_url, headers)

    if resp.status_code != 200:
        print(resp.content)
        return None

    return resp.json()

def get_call_demand(lat, lng, current_threshold=2, after_30m_threshold=2, max_num=3):
    header = {"Content-Type":"application/json"}
    body = {
        "center_lat" : lat,
        "center_lng": lng,
        "radius" : 16,
        "resolution" : 7
    }

    model_url = "https://airnd-wheel-gateway-dev.onkm.co.kr/call-demand-forecasting/predict"

    response = requests.post(model_url, json=body, headers=header)

    if response.status_code != 200:
        return None

    data = response.json()["predictions"]["data"]

    current_recommend_call = []
    call_after_30m_recommend_call = []
    for d in data:
        if d["call_now"] >= current_threshold:
            current_recommend_call.extend([d["hex_center"]["lat"], d["hex_center"]["lng"]])
        if d["call_after_30m"] > after_30m_threshold:
            call_after_30m_recommend_call.extend([d["hex_center"]["lat"], d["hex_center"]["lng"]])

    return current_recommend_call[:min(len(current_recommend_call), max_num*2)], call_after_30m_recommend_call[:min(len(call_after_30m_recommend_call), max_num*2)]

def parsing_address(address:dict):
    addresses = [ addr for addr in [address.get('name1', ''), address.get('name2', ''), address.get('name3', '')] if addr != '']
    lat_lng = f"({address['y']}, {address['x']})"
    return " ".join(addresses) + lat_lng

In [84]:
driver_lat, driver_lng = 37.508811, 127.040978
driver_address =parsing_address(get_address_by_lat_lng([driver_lat, driver_lng])[0])
current_recommend_call, call_after_30m_recommend_call = get_call_demand(driver_lat, driver_lng)
current_address = get_address_by_lat_lng(current_recommend_call)
call_after_30m_address = get_address_by_lat_lng(call_after_30m_recommend_call)

In [85]:
current_address_str_list = list(map(parsing_address, current_address))
call_after_30m_address_str_list = list(map(parsing_address, call_after_30m_address))

In [82]:
call_location_recommend_system_template = """
너는 "현재 기사님의 위치", "현재 수요가 많은 지역 리스트"와 "앞으로 30분 후에 수요가 많을 지역 리스트"를 입력으로 받아서 기사님들에게 해당 정보를 요약해주는 AI 모델이야.
너는 입력 받은 수요 지역 리스트 정보를 기반으로 현재 수요가 많은 지역, 앞으로 수요가 많을 지역들에 대해 설명해주어야 해.
이때 기사님들에게 추천해주는 거니까 강요하는 식으로 말하면 안 되고, 간격하고 공손한 말투로 대답해줘야 해
"""

call_location_recommend_question_example = """
- 현재 기사님 위치 : 서울특별시 강남구 역삼1동

- 현재 수요가 많은 지역 리스트 : ['서울특별시 강남구 논현1동', '서울특별시 종로구 종로1.2.3.4가동', '경기도 성남시 분당구 판교동']

- 앞으로 30분 후 수요가 많을 지역 리스트 : ['서울특별시 강남구 청담동', '서울특별시 송파구 가락본동', '서울특별시 광진구 광장동']
"""

call_location_recommend_answer_example = """
안녕하세요 기사님. 기사님의 위치를 기반으로 현재 콜이 많은 지역과 앞으로 콜이 많아질 지역을 추천해드릴게요

현재 기사님의 위치 "서울특별시 강남구 역삼1동"에서 가까운 지역 중에서 수요가 많은 지역은 동일한 강남구의 논현1동과 종로구의 종로, 경기도의 성남시 분당구입니다.

또한 앞으로 30분 후에 콜 예측을 수행했을 때에는, 강남구의 청담동, 송파구의 가락본동, 광진구의 광장동이 수요가 높을 것으로 예상됩니다.

논현1동은 현재 지역과 거리가 그리 멀지 않으니 해당 지역에 가서 대기해보셔도 좋을 것 같습니다.
또한 잠깐 다른 업무를 보셔야 한다면 업무를 보신 다음에 강남구의 청담동에 가보셔도 좋을 것 같습니다.
"""

user_input = """
- 현재 기사님 위치 : {driver_location}

- 현재 수요가 많은 지역 리스트 : {current_address_str_list}

- 앞으로 30분 후 수요가 많을 지역 리스트 : {call_after_30m_address_str_list}
"""

In [83]:
driver_address

'서울특별시 강남구 역삼1동(37.49542431718493, 127.03320108651666)'

In [89]:
import os
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = "https://kmt-oai-fr-37.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "7b214487cbbb49058ad896b129b1cafc"
os.environ["OPENAI_API_VERSION"] = "2023-07-01-preview"

In [90]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import SystemMessage, AIMessage
from langchain.schema import HumanMessage

chat = AzureChatOpenAI(azure_deployment="gpt-4")

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content=call_location_recommend_system_template),
        HumanMessage(content=call_location_recommend_question_example),
        AIMessage(content=call_location_recommend_answer_example),
        HumanMessage(content=user_input.format(
            driver_location=driver_address,
            current_address_str_list=current_address_str_list,
            call_after_30m_address_str_list=call_after_30m_address_str_list)),
    ]
)

chain = prompt | chat

In [96]:
model_resp = chain.invoke({})

In [101]:
driver_address

'서울특별시 강남구 역삼1동(37.49542431718493, 127.03320108651666)'

In [102]:
current_address_str_list

['서울특별시 강남구 도곡2동(37.483728401661516, 127.04638382177308)',
 '서울특별시 서초구 양재1동(37.47163933132787, 127.02672880394384)',
 '서울특별시 광진구 화양동(37.546476261142864, 127.07136761473575)']

In [103]:
call_after_30m_address_str_list

['서울특별시 강남구 역삼2동(37.49596751103195, 127.0468346777432)',
 '서울특별시 강남구 논현1동(37.51149226338117, 127.02855272024502)',
 '서울특별시 서초구 서초3동(37.48370806396488, 127.01200750943013)']

In [97]:
print(model_resp.content)

기사님, 현재 위치하고 계신 서울특별시 강남구 역삼1동 주변에서 수요가 많은 지역들에 대한 정보를 알려드리겠습니다.

현재로서는 강남구 도곡2동, 서초구 양재1동, 그리고 광진구 화양동 쪽에서 수요가 많이 발생하고 있습니다. 이 중에서 도곡2동과 양재1동은 기사님의 현재 위치와 가까우니 이 지역들을 고려해보실 수 있을 것 같습니다.

그리고 앞으로 30분 뒤에는 역삼2동, 논현1동, 서초3동에서 수요가 많을 것으로 예상됩니다. 특히 역삼2동은 현재 기사님의 위치와 매우 가깝기 때문에, 조금 이따가 고객 수요가 높아지는 시점에 맞춰서 이동하시면 좋을 것 같습니다.

기사님의 일정과 루트를 고려하여 효율적인 이동을 계획해보시는 것이 좋겠습니다. 안전 운행하시길 바랍니다.


In [26]:
call_df = pd.DataFrame(df)
df.to_csv("../data/h3_call_demand.csv")

---

In [28]:
call_df = pd.DataFrame(df)

In [37]:
call_df[call_df["call_now"]>1]

,h3res7,call_now,call_after_30m,call_after_60m,center_lat,center_lng
37,8730e0269ffffff,3,3,1,37.385205,127.092241
70,8730e0aedffffff,3,0,0,37.567390,126.832508
101,8730e1436ffffff,3,3,1,37.287498,127.230689
103,8730e0a60ffffff,3,0,0,37.682619,126.786795


In [38]:
call_df[call_df["call_after_30m"]>1]

,h3res7,call_now,call_after_30m,call_after_60m,center_lat,center_lng
0,8730e1ca3ffffff,0,3,1,37.500817,127.046889
1,8730e1ca2ffffff,0,2,3,37.493210,127.027914
2,8730e1c84ffffff,0,3,2,37.481047,127.045243
3,8730e1caeffffff,1,2,0,37.508423,127.065875
4,8730e1ca1ffffff,0,2,0,37.520588,127.048537
18,8730e1d89ffffff,0,2,1,37.564672,127.015495
21,8730e1d83ffffff,1,3,0,37.541805,126.958592
37,8730e0269ffffff,3,3,1,37.385205,127.092241
45,8730e1531ffffff,0,2,3,37.388162,127.147518
49,8730e1db5ffffff,0,3,3,37.570555,126.887660


In [39]:
call_df[call_df["call_after_60m"]>1]

,h3res7,call_now,call_after_30m,call_after_60m,center_lat,center_lng
1,8730e1ca2ffffff,0,2,3,37.493210,127.027914
2,8730e1c84ffffff,0,3,2,37.481047,127.045243
9,8730e1ca4ffffff,0,0,3,37.525134,127.012220
11,8730e1c13ffffff,1,0,2,37.547976,127.069184
22,8730e1d9cffffff,0,0,2,37.522047,126.956973
23,8730e036dffffff,0,0,2,37.482536,126.953740
26,8730e1d8dffffff,0,0,3,37.576818,126.998149
27,8730e1d8cffffff,1,1,2,37.569192,126.979175
30,8730e1c9affffff,1,1,3,37.404975,127.093901
36,8730e1d95ffffff,0,0,3,37.526560,126.920712
